In [1]:
%load_ext autoreload
%autoreload 2

In [53]:
import pandas as pd 
import numpy as np
from pomme.preparation_donnees_electorales import preparation_donnees_electorales_brutes
from functools import reduce 
from tqdm import tqdm

### Import des données - résultats électoraux

In [54]:
pres_dict = {'../db/electoral_data/raw/presidentielle/PR_2022_T2.csv': [2022,2],
'../db/electoral_data/raw/presidentielle/PR_2022_T1.csv':[ 2022,1],
'../db/electoral_data/raw/presidentielle/PR_2017_T2.csv':[2017,1],
'../db/electoral_data/raw/presidentielle/PR_2017_T1.csv':[2017,1]}

dfs_to_concat = []
for file, elec in tqdm(pres_dict.items()):
 
    tmp_df=pd.read_csv(file, sep=';', encoding='latin-1',header = None,skiprows=1)
    tmp_df = preparation_donnees_electorales_brutes(tmp_df, elec[1], cat_elec ='pres', annee=elec[0])
    dfs_to_concat.append(tmp_df)

leg_dict = {'../db/electoral_data/raw/legislative/LG_2022_T1.csv': [2022,1],
            '../db/electoral_data/raw/legislative/LG_2022_T2.csv':[2022,2],
            '../db/electoral_data/raw/legislative/LG_2017_T1.csv':[2017,1],
            '../db/electoral_data/raw/legislative/LG_2017_T2.csv':[2017,2]}

for file, elec in tqdm(leg_dict.items()):
  
    tmp_df=pd.read_csv(file, sep=";", encoding="latin-1", names = list(range(300)))
    tmp_df.dropna(axis=1, how='all', inplace=True)
    tmp_df = preparation_donnees_electorales_brutes(tmp_df, elec[1], cat_elec ='leg',annee=elec[0])
    dfs_to_concat.append(tmp_df)

  0%|          | 0/4 [00:00<?, ?it/s]/var/folders/rc/0g2xxgc57fq40kc5jwsm7h_c0000gn/T/ipykernel_51607/1877943886.py:9: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp_df=pd.read_csv(file, sep=';', encoding='latin-1',header = None,skiprows=1)
 25%|██▌       | 1/4 [00:00<00:02,  1.40it/s]/var/folders/rc/0g2xxgc57fq40kc5jwsm7h_c0000gn/T/ipykernel_51607/1877943886.py:9: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp_df=pd.read_csv(file, sep=';', encoding='latin-1',header = None,skiprows=1)
 50%|█████     | 2/4 [00:03<00:03,  1.89s/it]/var/folders/rc/0g2xxgc57fq40kc5jwsm7h_c0000gn/T/ipykernel_51607/1877943886.py:9: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  tmp_df=pd.read_csv(file, sep=';', encoding='latin-1',header = None,skiprows=1)
 75%|███████▌  | 3/4 [00:03<00:01,  1.24s/it]/var/folders/rc/0g2xxgc57fq40kc5j

In [55]:
cols_pres_12 = ['Tour',
'Code du département',
'Code de la commune',
'Libellé de la commune',
'Code de la circonscription',
'Code du canton',
'Code du b.vote',
'Inscrits',
'Votants',
'Exprimés',
'N°Panneau',
'Nom',
'Prénom',
'Nuance',
'Voix']

file='../db/electoral_data/raw/presidentielle/PR_2012_T1T2.csv'
pres_12_T1T2=pd.read_csv(file, sep=';', encoding='latin-1', header=None, skiprows=0)
pres_12_T1T2.columns =cols_pres_12
pres_12_T1T2['Election']='pres'
pres_12_T1T2['Annee']=2012
dfs_to_concat.append(pres_12_T1T2)

cols_leg_07= ['Tour',
'Code du département',
'Code de la commune',
'Libellé de la commune',
'Code du b.vote',
'Inscrits',
'Votants',
'Exprimés',
'N°Panneau',
'Nom',
'Prénom',
'Nuance',
'Voix']

file='../db/electoral_data/raw/legislative/LG_2007_T1T2.csv'
leg_07_T1T2=pd.read_csv(file, sep=';', encoding='latin-1', header=None, names=list(range(200)))
leg_07_T1T2.dropna(axis=1, how='all', inplace=True)
leg_07_T1T2.drop(list(range(18)), axis=0,inplace=True)
leg_07_T1T2.columns=cols_leg_07
leg_07_T1T2['Election']='leg'
leg_07_T1T2['Annee']=2007
dfs_to_concat.append(leg_07_T1T2)

/var/folders/rc/0g2xxgc57fq40kc5jwsm7h_c0000gn/T/ipykernel_51607/163409096.py:18: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  pres_12_T1T2=pd.read_csv(file, sep=';', encoding='latin-1', header=None, skiprows=0)
/var/folders/rc/0g2xxgc57fq40kc5jwsm7h_c0000gn/T/ipykernel_51607/163409096.py:39: DtypeWarning: Columns (0,1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  leg_07_T1T2=pd.read_csv(file, sep=';', encoding='latin-1', header=None, names=list(range(200)))


In [56]:
#Une fois tous les résultats récupérés, on les concatène dans un meme datfarame contenant tous les resultats
dfs_to_concat_cols=[df.columns for df in dfs_to_concat]
common_cols= list(reduce(set.intersection, map(set,dfs_to_concat_cols)))

resultats_concat= pd.DataFrame(columns= common_cols)
for result in dfs_to_concat : 
    resultats_concat = pd.concat([result,resultats_concat], axis = 0)

resultats_concat['Nom'] = resultats_concat['Nom'].fillna('NaN')
resultats_concat['Nom'] = resultats_concat['Nom'].apply(lambda x: x.replace('\x83', 'É'))

In [94]:
resultats_concat.columns

Index(['Tour', 'Code du département', 'Code de la commune',
       'Libellé de la commune', 'Code du b.vote', 'Inscrits', 'Votants',
       'Exprimés', 'N°Panneau', 'Nom', 'Prénom', 'Nuance', 'Voix', 'Election',
       'Annee', 'Code de la circonscription', 'Code du canton',
       'Libellé du département', 'Libellé de la circonscription',
       'Abstentions', '% Abs/Ins', '% Vot/Ins', 'Blancs', '% Blancs/Ins',
       '% Blancs/Vot', 'Nuls', '% Nuls/Ins', '% Nuls/Vot', '% Exp/Ins',
       '% Exp/Vot', '% Voix/Ins', '% Voix/Exp', 'Code INSEE'],
      dtype='object')

In [61]:
## On complète les nuances manquantes
nuances_manquantes = {'ARTHAUD' : "LO",
 'ROUSSEL' : "PC",
 'MACRON' : "LREM",
 'LASSALLE' : "LIOT",
 'LE PEN' : "RN",
 'ZEMMOUR' : "REC",
 'MÉLENCHON' : "LFI",
 'HIDALGO' : "PS",
 'JADOT' : "EELV",
 'PÉCRESSE' : "LR",
 'POUTOU' : "NPA",
 'DUPONT-AIGNAN' : "DLF",
 'HAMON' : "PS",
 'CHEMINADE' : "SP",
 'ASSELINEAU' : "UPR",
 'FILLON' : "LR",
 'JOLY':'EELV',
 'HOLLANDE' : "PS",
 "SARKOZY":"LR",
 "BAYROU":'MDM'
}

nuances_regroupees ={'LO' : "EXTG",
 'PC' : "EXTG",
 'LREM' : "CENTRE-DROIT",
 'LIOT':"CENTRE-DROIT",
 'RN' : "EXTD",
 'REC' : "EXTD",
 'LFI' : "EXTG",
 'PS' : "GCHE",
 'EELV' : "ECO",
 'LR' : "DRTE",
 'NPA' : "EXTG",
 'DLF' : "EXTD",
 'DXG' : "EXTG",
 'DVG' : "GCHE",
 'DIV' : "DIVERS",
 'REG' : "DIVERS",
 'ECO' : "ECO",
 'NUP' : "EXTG",
 'RDG' : "CENTRE-GCHE",
 'ENS' : "CENTRE-DROIT",
 'DVC' : "CENTRE",
 'DSV' : "EXTD",
 'DVD' : "DRTE",
 'UDI' : "CENTRE-DROIT",
 'DXD' : "EXTD",
 'SP' : "EXTD",
 'UPR' : "EXTD",
 'EXG' : "EXTG",
 'FI' : "EXTG",
 'COM': "EXTG",
 'SOC' : "GCHE",
 'MDM' : "CENTRE",
 'REM' : "CENTRE-DROIT",
 'FN' : "EXTD",
 'EXD' : "EXTD",
 'VEC' : "ECO",
 'UMP' : "DRTE",
 'AUT' : "DIVERS",
 'NCE' : "CENTRE-DROIT",
 'PRV' :"CENTRE-DROIT",
 'FG' : "EXTG",
 'CEN' : "CENTRE",
 'ALLI' : "CENTRE",
 'UDFD': "CENTRE-DROIT",
 'CPNT' : "DRTE",
 'MPF' : "EXTD",
 'MAJ': "DRTE"}

loc_echiquier = {'CENTRE-DROIT' : 1,
'ECO' : -1,
'GCHE' :-2,
'EXTD' :  3,
'DIVERS': 0,
'DRTE' :2,
'CENTRE-GCHE': -1,
'CENTRE':0,
'EXTG' : -3}


resultats_concat.Nuance=resultats_concat.Nom.map(nuances_manquantes).fillna(resultats_concat.Nuance)
resultats_concat.Nuance=resultats_concat.Nuance.map(nuances_regroupees)
resultats_concat.loc_nuance=resultats_concat.Nuance.map(loc_echiquier)

/var/folders/rc/0g2xxgc57fq40kc5jwsm7h_c0000gn/T/ipykernel_51607/1205006077.py:85: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  resultats_concat.loc_nuance=resultats_concat.Nuance.map(loc_echiquier)


In [62]:
resultats_concat=resultats_concat[resultats_concat['Code de la commune']!='Code de la commune'] # à voir pq mais les headers ont été concaténés ?

In [64]:
#on droppe les résultats de l'étranger, keeping row where Code du département is numeric 
resultats_concat=resultats_concat[resultats_concat['Code du département'].apply(lambda x: str(x).isnumeric())]

In [69]:
#on complète les codes pour retrouver le sbureaux de vote ensuite
resultats_concat["Code de la commune"].fillna(0, inplace=True)
resultats_concat["Code de la commune"].replace("nan", 0, inplace=True)
resultats_concat["Code de la commune"] = resultats_concat["Code de la commune"].apply(lambda  x : str(int(x)).zfill(3))
resultats_concat["Code du département"] = resultats_concat["Code du département"].apply(lambda  x : str(x).zfill(2))
resultats_concat["Code INSEE"]=resultats_concat["Code du département"]+ resultats_concat["Code de la commune"]
resultats_concat['Code du b.vote'] = resultats_concat['Code du b.vote'].apply(lambda x: str(x).zfill(4))

### Import des données - Bureaux de vote & Merge

In [163]:
file = "../db/bureaux_de_vote/bureaux-vote-france-2017.csv"
bureaux_de_vote = pd.read_csv(file, sep=";", encoding="utf-8")
bureaux_de_vote.dropna(subset=['Code Postal'], inplace=True)

In [164]:
bureaux_de_vote["Code INSEE"]=bureaux_de_vote["Code INSEE"].fillna(0).apply(lambda  x : str(int(x)).zfill(5))
bureaux_de_vote.loc[bureaux_de_vote.Ville=='Paris','Code INSEE']='75056'
resultats_concat['Code du b.vote'] = resultats_concat["Code du b.vote"].apply(lambda x : str(x).zfill(4))



In [159]:
# all_results['Code Postal'].apply(lambda  x : str(int(x)).zfill(5))

In [165]:
all_results = pd.merge(resultats_concat, bureaux_de_vote, how='left',  right_on=['Code INSEE','Code Bureau Vote'], left_on = ['Code INSEE', 'Code du b.vote'])

#Porblème avec le code INSEE PAris dans le fichier resulttats concat: il est à 75056 alors que dans le fichier bureau de vote, les codes insee sont renseignés
# bureaux_de_vote_paris = bureaux_de_vote[bureaux_de_vote.Ville =='Paris' ]


# bureaux_de_vote_paris['Code INSEE'] ='75056'
# all_results=pd.merge(all_results, bureaux_de_vote_paris,how='left',right_on=['Code INSEE','Code Bureau Vote'], left_on = ['Code INSEE', 'Code du b.vote'])

#all_results.to_csv('../db/results/all_results.csv')

In [166]:
del dfs_to_concat
del bureaux_de_vote

NameError: name 'dfs_to_concat' is not defined

In [167]:
#get nuances_regroupees.values() unique values as list 
resultats_concat[resultats_concat['Code du département']=='75']['Code INSEE'].unique()

array(['75056'], dtype=object)

In [168]:
bureaux_de_vote[bureaux_de_vote.Ville =='Paris' ]

,Bureau de vote,Code INSEE,Code Bureau Vote,Heure d'ouverture,Heure de fermeture,Code Postal,Ville,Nom Bureau Vote,Adresse,Adresse Substitution,Coordonnées,Département,uniq_bdv
1860,Bureau de vote n°55,75056,1655,8,20,75016.0,Paris,GYMNASE HENRY DE MONTHERLANT,32 BOULEVARD LANNES,32 BOULEVARD LANNES,"48.866985, 2.271843",75,GYMNASE HENRY DE MONTHERLANT - Paris
1861,Bureau de vote n°3,75056,1503,8,20,75015.0,Paris,LYCEE CAMILLE SEE,11 RUE LEON LHERMITTE,11 RUE LEON LHERMITTE,"48.842958, 2.296931",75,LYCEE CAMILLE SEE - Paris
1862,Bureau de vote n°67,75056,1567,8,20,75015.0,Paris,GROUPE D'ENSEIGNEMENT TECHNIQUE DU BATIMENT,15 RUE SAINT LAMBERT,15 RUE SAINT LAMBERT,"48.837765, 2.291519",75,GROUPE D'ENSEIGNEMENT TECHNIQUE DU BATIMENT - ...
1863,Bureau de vote n°5,75056,0305,8,20,75003.0,Paris,ECOLE ELEMENTAIRE,10 bis RUE DES QUATRE FILS,10 bis RUE DES QUATRE FILS,"48.860401, 2.360171",75,ECOLE ELEMENTAIRE - Paris
1864,Bureau de vote n°20,75056,1320,8,20,75013.0,Paris,ECOLE ELEMENTAIRE,3 RUE EMILE LEVASSOR,3 RUE EMILE LEVASSOR,"48.820888, 2.369042",75,ECOLE ELEMENTAIRE - Paris
...,...,...,...,...,...,...,...,...,...,...,...,...,...
63995,Bureau de vote n°15,75056,0615,8,20,75006.0,Paris,CENTRE ANDRE MALRAUX,112 RUE DE RENNES,112 RUE DE RENNES,"48.847869, 2.327529",75,CENTRE ANDRE MALRAUX - Paris
63996,Bureau de vote n°34,75056,1734,8,20,75017.0,Paris,ECOLE ELEMENTAIRE,23 AVENUE DE SAINT OUEN,23 AVENUE DE SAINT OUEN,"48.889046, 2.325925",75,ECOLE ELEMENTAIRE - Paris
63997,Bureau de vote n°45,75056,1645,8,20,75016.0,Paris,ECOLE MATERNELLE,1 RUE DU SERGENT MAGINOT,1 RUE DU SERGENT MAGINOT,"48.840539, 2.255537",75,ECOLE MATERNELLE - Paris
63998,Bureau de vote n°25,75056,1125,8,20,75011.0,Paris,ECOLE ELEMENTAIRE,1 RUE PIHET,1 RUE PIHET,"48.863455, 2.373882",75,ECOLE ELEMENTAIRE - Paris


In [ ]:
all_results.columns 

Index(['Tour', 'Code du département', 'Code de la commune',
       'Libellé de la commune', 'Code du b.vote', 'Inscrits', 'Votants',
       'Exprimés', 'N°Panneau', 'Nom', 'Prénom', 'Nuance', 'Voix', 'Election',
       'Annee', 'Code de la circonscription', 'Code du canton',
       'Libellé du département', 'Libellé de la circonscription',
       'Abstentions', '% Abs/Ins', '% Vot/Ins', 'Blancs', '% Blancs/Ins',
       '% Blancs/Vot', 'Nuls', '% Nuls/Ins', '% Nuls/Vot', '% Exp/Ins',
       '% Exp/Vot', '% Voix/Ins', '% Voix/Exp', 'Code INSEE',
       'Bureau de vote_x', 'Code Bureau Vote_x', 'Heure d'ouverture_x',
       'Heure de fermeture_x', 'Code Postal_x', 'Ville_x', 'Nom Bureau Vote_x',
       'Adresse_x', 'Adresse Substitution_x', 'Coordonnées_x', 'Département_x',
       'uniq_bdv_x', 'Bureau de vote_y', 'Code Bureau Vote_y',
       'Heure d'ouverture_y', 'Heure de fermeture_y', 'Code Postal_y',
       'Ville_y', 'Nom Bureau Vote_y', 'Adresse_y', 'Adresse Substitution_y',
      

In [169]:
#drop rows where Bureau de vote is NaN
all_results.dropna(subset=['Bureau de vote'], inplace=True)


In [170]:
all_results

,Tour,Code du département,Code de la commune,Libellé de la commune,Code du b.vote,Inscrits,Votants,Exprimés,N°Panneau,Nom,...,Heure d'ouverture,Heure de fermeture,Code Postal,Ville,Nom Bureau Vote,Adresse,Adresse Substitution,Coordonnées,Département,uniq_bdv
0,1,01,277,Nivollet-Montgriffon,0001,63.0,42.0,40.0,42.0,BAILLY,...,8.0,19.0,1230.0,Nivollet-Montgriffon,Mairie de Nivollet,NaN,Mairie de Nivollet,"45.992147, 5.444118",01,Mairie de Nivollet - Nivollet-Montgriffon
1,1,01,277,Nivollet-Montgriffon,0001,63.0,42.0,40.0,43.0,FRANQUET,...,8.0,19.0,1230.0,Nivollet-Montgriffon,Mairie de Nivollet,NaN,Mairie de Nivollet,"45.992147, 5.444118",01,Mairie de Nivollet - Nivollet-Montgriffon
2,1,01,277,Nivollet-Montgriffon,0001,63.0,42.0,40.0,46.0,ALBERT,...,8.0,19.0,1230.0,Nivollet-Montgriffon,Mairie de Nivollet,NaN,Mairie de Nivollet,"45.992147, 5.444118",01,Mairie de Nivollet - Nivollet-Montgriffon
3,1,01,277,Nivollet-Montgriffon,0001,63.0,42.0,40.0,47.0,MARMORAT,...,8.0,19.0,1230.0,Nivollet-Montgriffon,Mairie de Nivollet,NaN,Mairie de Nivollet,"45.992147, 5.444118",01,Mairie de Nivollet - Nivollet-Montgriffon
4,1,01,277,Nivollet-Montgriffon,0002,49.0,33.0,33.0,7.0,FAURE,...,8.0,19.0,1230.0,Nivollet-Montgriffon,Salle polyvalente de Montgriffon,NaN,Salle polyvalente de Montgriffon,"46.002213, 5.470505",01,Salle polyvalente de Montgriffon - Nivollet-M...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7142330,2,95,680,Villiers-le-Bel,0011,1227,731,676,NaN,LE PEN,...,8.0,20.0,95400.0,Villiers-le-Bel,École Marie Curie,45 Rue de la Republique,45 Rue de la Republique,"49.006733, 2.387895",95,École Marie Curie - Villiers-le-Bel
7142331,2,95,680,Villiers-le-Bel,0012,912,585,535,NaN,LE PEN,...,8.0,20.0,95400.0,Villiers-le-Bel,Groupe Scolaire Jean Jaurès,Rue Anadou Hampate BA,Rue Anadou Hampate BA,"49.001856, 2.415228",95,Groupe Scolaire Jean Jaurès - Villiers-le-Bel
7142332,2,95,680,Villiers-le-Bel,0013,682,375,340,NaN,LE PEN,...,8.0,20.0,95400.0,Villiers-le-Bel,École Primaire Ferdinand Buisson,Rue Jean Bullant,Rue Jean Bullant,"49.003734, 2.395569",95,École Primaire Ferdinand Buisson - Villiers-l...
7142333,2,95,682,Villiers-le-Sec,0001,110,84,80,NaN,LE PEN,...,8.0,20.0,95720.0,Villiers-le-Sec,Mairie annexe,6 rue de Paris,6 rue de Paris,"49.072383, 2.390203",95,Mairie annexe - Villiers-le-Sec


In [171]:
def try_int(x):
    try:
        return str(int(x)).zfill(1)
    except ValueError:
        return x


all_results['Code du b.vote'] = all_results['Code du b.vote'].apply(try_int)
#all_results['Code Postal'] = all_results['Code Postal'].astype(int).astype(str).str.zfill(5)


In [172]:
all_results['id_bv'] = (all_results['Code INSEE']+'_'+all_results['Code du b.vote'])

In [173]:
all_results[all_results['Code du département']=='75']


,Tour,Code du département,Code de la commune,Libellé de la commune,Code du b.vote,Inscrits,Votants,Exprimés,N°Panneau,Nom,...,Heure de fermeture,Code Postal,Ville,Nom Bureau Vote,Adresse,Adresse Substitution,Coordonnées,Département,uniq_bdv,id_bv
644147,1,75,056,Paris,101,1104.0,731.0,725.0,122.0,BILLARD,...,20.0,75001.0,Paris,MAIRIE D'ARRONDISSEMENT,4 PLACE DU LOUVRE,4 PLACE DU LOUVRE,"48.860079, 2.341061",75,MAIRIE D'ARRONDISSEMENT - Paris,75056_101
644148,1,75,056,Paris,101,1104.0,731.0,725.0,124.0,DELCHER,...,20.0,75001.0,Paris,MAIRIE D'ARRONDISSEMENT,4 PLACE DU LOUVRE,4 PLACE DU LOUVRE,"48.860079, 2.341061",75,MAIRIE D'ARRONDISSEMENT - Paris,75056_101
644149,1,75,056,Paris,101,1104.0,731.0,725.0,125.0,VERON,...,20.0,75001.0,Paris,MAIRIE D'ARRONDISSEMENT,4 PLACE DU LOUVRE,4 PLACE DU LOUVRE,"48.860079, 2.341061",75,MAIRIE D'ARRONDISSEMENT - Paris,75056_101
644150,1,75,056,Paris,101,1104.0,731.0,725.0,127.0,DE PONS,...,20.0,75001.0,Paris,MAIRIE D'ARRONDISSEMENT,4 PLACE DU LOUVRE,4 PLACE DU LOUVRE,"48.860079, 2.341061",75,MAIRIE D'ARRONDISSEMENT - Paris,75056_101
644151,1,75,056,Paris,101,1104.0,731.0,725.0,129.0,HANRYON,...,20.0,75001.0,Paris,MAIRIE D'ARRONDISSEMENT,4 PLACE DU LOUVRE,4 PLACE DU LOUVRE,"48.860079, 2.341061",75,MAIRIE D'ARRONDISSEMENT - Paris,75056_101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7128022,2,75,056,Paris,1836,1501,1185,1127,NaN,LE PEN,...,20.0,75018.0,Paris,ECOLE MATERNELLE,4 SQUARE LAMARCK,4 SQUARE LAMARCK,"48.890833, 2.335818",75,ECOLE MATERNELLE - Paris,75056_1836
7128023,2,75,056,Paris,1837,1606,1250,1187,NaN,LE PEN,...,20.0,75018.0,Paris,ECOLE ELEMENTAIRE,65 RUE DAMREMONT,65 RUE DAMREMONT,"48.891953, 2.335055",75,ECOLE ELEMENTAIRE - Paris,75056_1837
7128024,2,75,056,Paris,1849,1170,660,603,NaN,LE PEN,...,20.0,75018.0,Paris,ECOLE MATERNELLE,2 PLACE FRANCOISE DORLEAC,2 PLACE FRANCOISE DORLEAC,"48.900308, 2.337173",75,ECOLE MATERNELLE - Paris,75056_1849
7128025,2,75,056,Paris,1850,1165,652,590,NaN,LE PEN,...,20.0,75018.0,Paris,ECOLE ELEMENTAIRE,19 RUE FERNAND LABORI,19 RUE FERNAND LABORI,"48.899112, 2.342241",75,ECOLE ELEMENTAIRE - Paris,75056_1850


In [174]:
all_results.to_csv('../db/results/all_results.csv')

In [175]:
all_results.columns

Index(['Tour', 'Code du département', 'Code de la commune',
       'Libellé de la commune', 'Code du b.vote', 'Inscrits', 'Votants',
       'Exprimés', 'N°Panneau', 'Nom', 'Prénom', 'Nuance', 'Voix', 'Election',
       'Annee', 'Code de la circonscription', 'Code du canton',
       'Libellé du département', 'Libellé de la circonscription',
       'Abstentions', '% Abs/Ins', '% Vot/Ins', 'Blancs', '% Blancs/Ins',
       '% Blancs/Vot', 'Nuls', '% Nuls/Ins', '% Nuls/Vot', '% Exp/Ins',
       '% Exp/Vot', '% Voix/Ins', '% Voix/Exp', 'Code INSEE', 'Bureau de vote',
       'Code Bureau Vote', 'Heure d'ouverture', 'Heure de fermeture',
       'Code Postal', 'Ville', 'Nom Bureau Vote', 'Adresse',
       'Adresse Substitution', 'Coordonnées', 'Département', 'uniq_bdv',
       'id_bv'],
      dtype='object')

In [176]:
all_results = pd.read_csv('../db/results/all_results.csv')

/var/folders/rc/0g2xxgc57fq40kc5jwsm7h_c0000gn/T/ipykernel_51607/2411600720.py:1: DtypeWarning: Columns (10,11,12,18,19,21,22,24,25,27,28,29,30,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  all_results = pd.read_csv('../db/results/all_results.csv')


In [177]:
#replace nan by NaN before replacing \x83 by E in the column Nom


In [178]:
[ele for ele in all_results['Nom'].unique() if type(ele)==float]

[nan]

In [179]:
all_results[all_results['Code Postal'].astype(str).str.startswith('94')][['Election','Tour','Annee']].drop_duplicates()

,Election,Tour,Annee
43300,leg,1,2007
754977,leg,2,2007
873207,pres,1,2012
1436471,pres,2,2012
1551581,leg,2,2017
1730978,leg,1,2017
3357977,leg,2,2022
3539693,leg,1,2022
4928621,pres,1,2017
5604570,pres,1,2022
